In [78]:
from pymongo import MongoClient
import json
import pandas as pd
import numpy as np


client = MongoClient('mongodb://localhost:27017/')
db = client.companies


In [79]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies')

In [80]:
filtered_companies = db.filtered_companies.find()
companies_dislike = db.companies_dislike.find()

In [81]:
filtered_companies = pd.DataFrame(list(filtered_companies))
companies_dislike = pd.DataFrame(list(companies_dislike))

In [82]:
filtered_companies.head(3)

,_id,category_code,deadpooled_year,description,founded_year,funding_rounds,ipo,lat,lng,name,number_of_employees,offices,oficina_principal,tag_list,totalOffices,total_money_raised,total_money_raised_USD
0,5d2a34ba73f7ac655759e983,software,None,Private Academic Networks,2008,"[{'funded_day': 1, 'source_url': '', 'raised_c...",None,37.782263,-122.392142,GoingOn,40.0,"[{'address2': '12th Floor', 'description': 'Go...","{'type': 'Point', 'coordinates': [-122.392142,...","socialnetworking, enterprise, social-media, so...",1,$8.5M,8500000
1,5d2a34ba73f7ac655759e984,games_video,None,Digital Entertainment and Media Platform,2008,"[{'funded_day': 1, 'source_url': 'http://www.d...",None,37.781265,-122.393229,Crunchyroll,50.0,"[{'address2': '', 'description': '', 'address1...","{'type': 'Point', 'coordinates': [-122.393229,...","crunchyroll, anime, manga, video, gaming, musi...",1,$4.8M,4800000
2,5d2a34ba73f7ac655759e985,enterprise,None,NoSQL database managed service,2008,"[{'funded_day': None, 'source_url': '', 'raise...",None,42.358920,-71.057810,Cloudant,70.0,"[{'address2': '', 'description': '', 'address1...","{'type': 'Point', 'coordinates': [-71.05781, 4...","cloud-computing, big-data, databases, software...",1,$18.3M,18300000


In [83]:
companies_dislike.head()

,_id,category_code,deadpooled_year,description,founded_year,funding_rounds,ipo,lat,lng,name,number_of_employees,offices,oficina_principal,tag_list,totalOffices,total_money_raised
0,5d29ec6dabbcf27c969c0195,web,None,None,1999,[],None,37.506885,-122.247573,Postini,NaN,"[{'description': None, 'address2': None, 'addr...","{'type': 'Point', 'coordinates': [-122.247573,...",,1,$0
1,5d29ec6dabbcf27c969c0196,web,None,Geneology social network site,2006,"[{'funded_day': 1, 'source_url': '', 'round_co...",None,34.090368,-118.393064,Geni,18.0,"[{'description': 'Headquarters', 'address2': '...","{'type': 'Point', 'coordinates': [-118.393064,...","geni, geneology, social, family, genealogy",1,$16.5M
2,5d29ec6dabbcf27c969c0197,games_video,None,None,2006,"[{'funded_day': 9, 'source_url': 'http://www.l...",None,40.746497,-74.009447,Joost,0.0,"[{'description': '', 'address2': '', 'address1...","{'type': 'Point', 'coordinates': [-74.0094471,...","iptv, babelgum, television, video, thevenicepr...",1,$45M
3,5d29ec6dabbcf27c969c0198,news,None,user driven social content website,2004,"[{'funded_day': 1, 'source_url': 'http://www.m...",None,37.764726,-122.394523,Digg,60.0,"[{'description': None, 'address2': '', 'addres...","{'type': 'Point', 'coordinates': [-122.394523,...","community, social, news, bookmark, digg, techn...",1,$45M
4,5d29ec6dabbcf27c969c0199,news,None,,1986,[],"{'stock_symbol': 'NYSE:CBS', 'pub_day': None, ...",37.269175,-119.306607,CBS,NaN,"[{'description': None, 'address2': '', 'addres...","{'type': 'Point', 'coordinates': [-119.306607,...",None,1,$0


In [84]:
names_companies_dislike = list(companies_dislike['name'])

### Función para buscar las empresas que están en un radio determinado 

In [85]:
def findNear(geopoint, radio_max_meters=2000):
    return db.filtered_companies.find({"oficina_principal":{"$near":{
           "$geometry": geopoint,
           "$maxDistance": radio_max_meters,
         }
       }
    })


radio_max_meters=2000
num_offices = pd.DataFrame()

In [86]:
punto = filtered_companies['oficina_principal'][12]
punto

{'coordinates': [-122.397759, 37.787183], 'type': 'Point'}

In [87]:
lst = list(findNear(punto, radio_max_meters))
len(lst)

26

In [88]:
print(lst[0]['name'])

LiveRail


In [89]:
filtered_companies['unlike_offices'] = np.nan
filtered_companies.columns

Index(['_id', 'category_code', 'deadpooled_year', 'description',
       'founded_year', 'funding_rounds', 'ipo', 'lat', 'lng', 'name',
       'number_of_employees', 'offices', 'oficina_principal', 'tag_list',
       'totalOffices', 'total_money_raised', 'total_money_raised_USD',
       'unlike_offices'],
      dtype='object')

In [90]:
# VER 2

lenn = 0
for index_, data_ in filtered_companies.iterrows():
    lenn+=1
    #print('company %s' % (lenn))
    lst = list(findNear(data_['oficina_principal'], radio_max_meters))
    num = len(lst)
    lst_of_company_names = []   
    enemy_near = False
    for x in lst:
        if (x['name'] in names_companies_dislike):
            enemy_near = True
            break
    
    if(enemy_near == False):
        filtered_companies.loc[filtered_companies['name'] == data_['name'], "unlike_offices"] = 'False'
    
    else:
        filtered_companies.loc[filtered_companies['name'] == data_['name'], "unlike_offices"] = 'True'
        
filtered_companies.head(3)   

,_id,category_code,deadpooled_year,description,founded_year,funding_rounds,ipo,lat,lng,name,number_of_employees,offices,oficina_principal,tag_list,totalOffices,total_money_raised,total_money_raised_USD,unlike_offices
0,5d2a34ba73f7ac655759e983,software,None,Private Academic Networks,2008,"[{'funded_day': 1, 'source_url': '', 'raised_c...",None,37.782263,-122.392142,GoingOn,40.0,"[{'address2': '12th Floor', 'description': 'Go...","{'type': 'Point', 'coordinates': [-122.392142,...","socialnetworking, enterprise, social-media, so...",1,$8.5M,8500000,False
1,5d2a34ba73f7ac655759e984,games_video,None,Digital Entertainment and Media Platform,2008,"[{'funded_day': 1, 'source_url': 'http://www.d...",None,37.781265,-122.393229,Crunchyroll,50.0,"[{'address2': '', 'description': '', 'address1...","{'type': 'Point', 'coordinates': [-122.393229,...","crunchyroll, anime, manga, video, gaming, musi...",1,$4.8M,4800000,False
2,5d2a34ba73f7ac655759e985,enterprise,None,NoSQL database managed service,2008,"[{'funded_day': None, 'source_url': '', 'raise...",None,42.358920,-71.057810,Cloudant,70.0,"[{'address2': '', 'description': '', 'address1...","{'type': 'Point', 'coordinates': [-71.05781, 4...","cloud-computing, big-data, databases, software...",1,$18.3M,18300000,False


In [91]:
filtered_companies["unlike_offices"].value_counts()

False    223
Name: unlike_offices, dtype: int64

#### Aqui hacer un ciclo for con la función dentro para que nos diga cuantas oficinas hay cerca de nuestras posibles oficinas

In [92]:
lst_of_num_companies =[]
for comp in filtered_companies['oficina_principal']:
    lst_of_num_companies.append(len(list(findNear(comp, radio_max_meters))))


filtered_companies['offices_near']=lst_of_num_companies

In [93]:
definitive_companies = filtered_companies.sort_values('offices_near', ascending=False)

In [94]:
definitive_companies.head()


,_id,category_code,deadpooled_year,description,founded_year,funding_rounds,ipo,lat,lng,name,number_of_employees,offices,oficina_principal,tag_list,totalOffices,total_money_raised,total_money_raised_USD,unlike_offices,offices_near
71,5d2a34ba73f7ac655759e9ca,ecommerce,None,Spa and Well-being social network,2008,"[{'funded_day': 11, 'source_url': 'http://vent...",None,37.786910,-122.404403,InnerRewards,9.0,"[{'address2': 'Floor 6', 'description': 'West ...","{'type': 'Point', 'coordinates': [-122.404403,...","wellness, women, social-networking, spas, ecom...",2,$1M,1000000,False,33
58,5d2a34ba73f7ac655759e9bd,ecommerce,None,Spa and Well-being social network,2008,"[{'funded_day': 11, 'source_url': 'http://vent...",None,37.786910,-122.404403,InnerRewards,9.0,"[{'address2': 'Floor 6', 'description': 'West ...","{'type': 'Point', 'coordinates': [-122.404403,...","wellness, women, social-networking, spas, ecom...",2,$1M,1000000,False,33
187,5d2a34ba73f7ac655759ea3e,biotech,None,Cancer management,2008,"[{'funded_day': 6, 'source_url': 'http://sec.g...",None,37.787131,-122.404107,ACT Biotech,NaN,"[{'address2': 'Suite 650', 'description': '', ...","{'type': 'Point', 'coordinates': [-122.4041075...","biotech, development, cancer, drugs, medical, ...",1,$27M,27000000,False,33
164,5d2a34ba73f7ac655759ea27,enterprise,None,Social Enterprise Internet Technology,2008,"[{'funded_day': 1, 'source_url': 'http://blog....",None,37.778991,-122.401803,Samasource,25.0,"[{'address2': 'Suite 301', 'description': 'Hea...","{'type': 'Point', 'coordinates': [-122.401803,...","outsourcing, crowdsourcing, non-profit-outsour...",1,$1.21M,1200000,False,32
113,5d2a34ba73f7ac655759e9f4,web,None,Social Recruiting and Job Search Site,2008,"[{'funded_day': None, 'source_url': 'http://ww...",None,37.788796,-122.409710,KODA,15.0,"[{'address2': '#991', 'description': 'San Fran...","{'type': 'Point', 'coordinates': [-122.40971, ...",,2,$4.5M,4500000,False,32


In [95]:
definitive_companies.to_json('./definitive_companies.json', orient='records', default_handler=str)